In [5]:
import numpy as np
from astropy.io import fits
import pandas as pd
from datetime import datetime, timedelta

In [2]:
BAK_dict = {
'USER': ['Andreas Ramsli','Username of creator of this fits file'], #<str>
'T0TIME':['2020-12-27T15:14:06.854','Trigger time UT format=YYYY-MM-DDThh:mm:ss.fff'], #<str> 
'TSTART': [-0.9,'[s] Start time of background spectrum accumulation'], #<float> [s] Start time of spectrum accumulation relative to T0TIME
'TSTOP': [-0.3,'End time of background spectrum accumulation'], #<float> [s]
'EXPOSURE': [0.6,'[s] Integration time in seconds for the BAK data'], #<float> 
'OBJECT': ['GRB201227A','Name of the observed object'], #<str> 
'RA_OBJ': [170.121,'[deg] RA of source'], #<float> 
'DEC_OBJ':  [-73.613,'[deg] Dec of source'], #<float>
'FILENAME': ['example.bak','Backgorund filename for fits file'] #<str>  ('example.bak')
}
#More keys can be declated. Set generatePHA(see_template=True) to view avalible keys,values & comments from the template HDR's

In [1]:
def generateBAK(BAK_dict=None, bakCounts=None, stat_err=None, bin_lo=None, bin_hi=None, see_headers=False):
    """
    function for generating BAK file for ASIM data.
    BAK_dict <dict>, dictionary containing keywords and values for headers. Possible to set new keywords,values and comments by declaring them
    as 'KEY': ['VALUE','COMMENT'] in the BAK_dict.
    
    bin_lo: <np.array>, array containing the E_MIN boundaries for the channels
    bin_hi: <np.array>, array containing the E_MAX boundaries for the channels
    bakCounts: <np.array>, array containing the background counts <int> in each channel
    stat_err: <np.array>, array containing the poission error <float> for each count
    see_headers: <bool>, wheter to print the template HDR's. If true nothing will be returned
    """
    
    #Creating template for PRIMARY,SPECTRUM & EBOUNDS HDR
    #----------------------------------------------------------------
    primaryDictHDR = {'SIMPLE': [True, 'conforms to FITS standard'],
    'BITPIX': [8, 'array data type'],
    'NAXIS': [0, 'number of array dimensions'],
    'EXTEND': [True, ''],
    'CREATOR': ['ASIM Fits v1.0.0', 'Program name. Written by Andreas Ramsli'],
    'FILETYPE': ['BACKGROUND', 'Name for this type of FITS file'],
    'TELESCOP': ['ASIM', 'Name of mission/satellite'],
    'INSTRUME': ['MXGS: HED', 'Specific instrument used for observation'],
    'ORIGIN': ['University of Bergen', 'Name of institution making file'],
    'DATE': ['none','file creation date [YYYY-MM-DDThh:mm:ss UT]'],
    'TSTART': ['none', '[s] TSTART given relative to T0'],
    'TSTOP': ['none', '[s] TSTOP given relative to T0'],
    'EXPOSURE': ['none', '[s] Integration time in seconds for the PHA data'],
    'FILENAME': ['none', 'Name of this file'],
    'OBJECT': ['none', 'Burst name in standard format, GRB yymmdd'],
    'RADECSYS': ['FK5', 'Stellar reference frame'],
    'EQUINOX': [2000.0, 'Equinox for RA and Dec'],
    'RA_OBJ': ['none', '[deg] Calculated RA of source'],
    'DEC_OBJ': ['none', '[deg] Calculated Dec of source'],
    'T0TIME': ['none', 'Trigger time given as YYYY-MM-DDThh:mm:ss.fff'],
    'USER':['none','Username of creator of this fits file']}

    bakDictHDR = {'XTENSION': ['BINTABLE', 'binary table extension'],
    'BITPIX': [8, 'array data type'],
    'NAXIS': [2, 'number of array dimensions'],
    'NAXIS1': [10, 'length of dimension 1'],
    'NAXIS2': [40, 'length of dimension 2'],
    'PCOUNT': [0, 'number of group parameters'],
    'GCOUNT': [1, 'number of groups'],
    'TFIELDS': [3, 'number of table fields'],
    'ANCRFILE': ['none', 'ancillary response'],
    'AREASCAL': [1.0, 'Area scaling factor, no field AREASCAL'],
    'BACKFILE': ['none', 'Background FITS file'],
    'BACKSCAL': [1.0, 'Background scale factor, no field BACKSCAL'],
    'CHANTYPE': ['PI', 'channel type [PHA, PI etc]'],
    'CORRFILE': ['NONE', 'Correlation FITS file'],
    'CORRSCAL': [0.0, 'Correlation scale factor'],
    'CREATOR': ['ASIM fits v1.0.0',
    'Program name and version. By Andreas Ramsli'],
    'DATAMODE': ['PH', 'Datamode'],
    'DATE': ['none', 'File creation date [UT]'],
    'DATE_END': ['none', 'Date observations ended (yyyy-mm-dd)'],
    'DATE_OBS': ['none', 'Date observations were made [yyyy-mm-dd]'],
    'DETCHANS': [40, 'Total number of detector channels available'],
    'EQUINOX': [2000.0, 'Equinox of celestial coord system'],
    'EXPOSURE': ['none', '[s] Integration time in seconds for the PHA data'],
    'EXTNAME': ['SPECTRUM', 'Extension name'],
    'FILTER': ['none', 'Instrument filter in use'],
    'HDUCLAS1': ['SPECTRUM', 'PHA dataset'],
    'HDUCLAS2': ['BKG', 'indicating the data stored'],
    'HUDCLAS3': ['COUNT', 'indicating further the data stored'],               
    'HDUCLAS4': ['TYPE:I', 'indicating whether this is a type I or II exten'],
    'HDUCLASS': ['OGIP', 'format conforms to OGIP standard'],
    'HDUVERS': ['1.3.0', 'Version of format (OGIP memo OGIP-92-007)'],
    'INSTRUME': ['MXGS: HED', 'Instrument name'],
    'OBJECT': ['none', 'Name of observed object'],
    'ORIGIN': ['Univeristy of Bergen',
    'Name of institution making the fits file'],
    'POISSERR': [False, 'Set to false since stat_err column used'],
    'QUALITY': [0, 'no data quality information specified'],
    'GROUPING': [0, 'No special grouping applied'],
    'RADECSYS': ['FK5', 'coord frame used for EQUINOX'],
    'RESPFILE': ['none', 'redistribution matrix'],
    'SYS_ERR': [0, 'no systematic error specified'],
    'TELESCOP': ['ASIM', 'Telescope or mission name'],
    'TIME_END': ['none', 'Time observations were made (hh:mm:ss)'],
    'TIME_OBS': ['none', 'Time observations were made (hh:mm:ss)'],
    'TLMAX1': [40, 'Highest legal channel number'],
    'TLMIN1': [1, 'Lowest legal channel number'],
    'TOTCTS': ['none', 'Total counts in spectrum'],
    'T0TIME': ['none', 'Trigger time UT (YYYY-MM-DDThh:mm:ss.fff)'],
    'TSTART': ['none', '[s] Start time of spectrum accumulation relativ'],
    'TSTOP': ['none', '[s] End time of spectrum accumulation relative'],
    'COMMENT': ['TSTART/STOP is given relative to the trigger time (T0TIME)', ''],
    'USER': ['none ', 'Username of creator of this fits file'],
    'XFLT0001': ['none', 'XSPEC selection filter description'],
    'TTYPE1': ['CHANNEL', ''],
    'TFORM1': ['1I', ''],
    'TTYPE2': ['COUNTS', ''],
    'TFORM2': ['1E', ''],
    'TTYPE3': ['STAT_ERR', ''],
    'TFORM3': ['1E', ''],
    'RA_OBJ': ['', '[deg] Calculated RA of source'],
    'DEC_OBJ': ['', '[deg] Calculated Dec of source']}

    eboundsDictHDR = {'XTENSION': ['BINTABLE', 'binary table extension'],
    'BITPIX': [8, 'Required value'],
    'NAXIS': [2, 'Required value'],
    'NAXIS1': [10, 'Number of bytes per row'],
    'NAXIS2': [40, 'Number of rows'],
    'PCOUNT': [0, 'Normally 0 [no varying arrays]'],
    'GCOUNT': [1, 'Required value'],
    'TFIELDS': [3, 'Number of columns in table'],
    'EXTNAME': ['EBOUNDS', 'Extension name'],
    'TELESCOP': ['ASIM', 'Telescope or mission name'],
    'INSTRUME': ['MXGS: HED', 'Instrument name'],
    'DATE': ['none', 'Creation date'],
    'DETCHANS': [40, 'Total number of detector channels available'],
    'CHANTYPE': ['PHA', 'Channel type'],
    'TUNIT2': ['keV', 'Unit for this column'],
    'TUNIT3': ['keV', 'Unit for this column'],
    'TLMIN1': [1, 'Lowest legal channel number'],
    'TLMAX': [40, 'Highest legal channel number'],
    'EXTVER': [1, 'Version number of this extensio'],
    'HDUCLASS': ['OGIP', 'Required for RMF'],
    'HDUVERS': ['1.2.1', '(OGIP memo OGIP-92-007)'],
    'HDUCLAS1': ['RESPONSE', 'Typically found in RMF files'],
    'HDUCLAS2': ['EBOUNDS', 'Bounds for energybins'],
    'TFORM1': ['I', ''],
    'TFORM2': ['E', ''],
    'TFORM3': ['E', ''],
    'TTYPE1': ['CHANNEL', ''],
    'TTYPE2': ['E_MIN', '[keV] minimum energy of channel n'],
    'TTYPE3': ['E_MAX', '[keV] max energy of channel n']} 
    
    
    #Setting KEYWORDS in dict based on input
    #--------------------------------------------------------------------------------
    for key in BAK_dict.keys():
        try:
            primaryDictHDR[key][0] = BAK_dict[key]
            bakDictHDR[key][0] = BAK_dict[key]
            eboundsDictHDR[key][0] = BAK_dict[key]
        except KeyError: #creating new keys if they are not in the template dict
            if len(BAK_dict[key])<=1 or len(BAK_dict[key])>2:
                return(print("Value and comment must be set for the keyword '{}' as a list".format(key)))
            else:
                primaryDictHDR[key] = [BAK_dict[key][0],BAK_dict[key][1]]
                bakDictHDR[key] = [BAK_dict[key][0],BAK_dict[key][1]]
                eboundsDictHDR[key] =[BAK_dict[key][0],BAK_dict[key][1]]
                
    file_creation_date = datetime.isoformat(datetime.utcnow()) #setting file creation date to UTC now
    primaryDictHDR['DATE'][0], bakDictHDR['DATE'][0], eboundsDictHDR['DATE'][0] = file_creation_date,file_creation_date,file_creation_date
    
    #calculating the DATE_OBS,DATE_END,TIME_OBS,TIME_END from input
    try:
        T0TIME = datetime.fromisoformat(BAK_dict['T0TIME'][0])
        OBS_START = T0TIME + timedelta(seconds=BAK_dict['TSTART'][0])
        OBS_END = T0TIME + timedelta(seconds=BAK_dict['TSTOP'][0])
        
        bakDictHDR['DATE_OBS'][0] = OBS_START.strftime('%Y-%m-%d')
        bakDictHDR['TIME_OBS'][0] = OBS_START.strftime('%H:%M:%S')
        bakDictHDR['DATE_END'][0] = OBS_END.strftime('%Y-%m-%d')
        bakDictHDR['TIME_END'][0] = OBS_END.strftime('%H:%M:%S')
        
    except KeyError:
        return(print('T0TIME must be set in BAK_dict'))
    
    bakDictHDR['TOTCTS'][0] = np.sum(bakCounts) #summing all counts in array and setting value
    
    
    #Setting headers
    #----------------------------------------------------------------------------------
    primaryHDR,bakHDR, eboundsHDR = fits.Header(),fits.Header(),fits.Header()
    
    
    for row in primaryDictHDR.items():
        primaryHDR[row[0]] = row[1][0],row[1][1]
    for row in bakDictHDR.items():
        bakHDR[row[0]] = row[1][0],row[1][1]
    for row in eboundsDictHDR.items():
        eboundsHDR[row[0]] = row[1][0],row[1][1]
        
    if see_headers==True:
        print("PrimaryHDR: \n", primaryHDR.cards)
        print("BakHDR: \n",bakHDR.cards)
        print("EboundsHDR: \n",eboundsHDR.cards)
    else:
        primaryHDU = fits.PrimaryHDU(header=primaryHDR) #PrimaryHDU
        
        if not np.all([len(bakCounts),len(bin_lo),len(bin_hi),len(stat_err)]) == True:
            return(print("Length of input arrays are not the same"))
        
        #Creating table for SPECTRUM HDU file
        #------------------------------------
        channelColumn = fits.Column(name=bakHDR.cards['TTYPE1'][1],format=bakHDR.cards['TFORM1'][1], array=np.arange(1,len(bakCounts)+1))
        bakCountsColumn = fits.Column(name=bakHDR.cards['TTYPE2'][1],format=bakHDR.cards['TFORM2'][1], array=bakCounts)
        statErrColumn = fits.Column(name=bakHDR.cards['TTYPE3'][1],format=bakHDR.cards['TFORM3'][1], array=stat_err)
        bakTableHDU = fits.BinTableHDU.from_columns(columns=[channelColumn,bakCountsColumn,statErrColumn],header=bakHDR) 
          
        
        #Creating table for EBOUNDS HDU file
        #-----------------------------------
        eboundschannelColumn = fits.Column(name=eboundsHDR.cards['TTYPE1'][1],format=eboundsHDR.cards['TFORM1'][1], array=np.arange(1,len(bin_lo)+1))
        eboundMinColumn = fits.Column(name=eboundsHDR.cards['TTYPE2'][1],format=eboundsHDR.cards['TFORM2'][1], array=bin_lo)
        eboundMaxColumn = fits.Column(name=eboundsHDR.cards['TTYPE3'][1],format=eboundsHDR.cards['TFORM3'][1], array=bin_hi)
        eboundsTableHDU = fits.BinTableHDU.from_columns(columns=[channelColumn,eboundMinColumn,eboundMaxColumn],header=eboundsHDR) 
        
        #Writing to fits file
        #-----------------------------------
        HDUL = fits.HDUList([primaryHDU,bakTableHDU,eboundsTableHDU])
        try:
            HDUL.writeto(primaryHDR.cards['FILENAME'][1],checksum=True)
        except KeyError:
            print("FILENAME keyword must be set in input dict")
